# Connection and Message Security

For mutual TLS, FLARE's message security works as follows: the server and client authenticate each other when establishing direct connections. This ensures that only clients with the correct startup kits can connect to the server.

However mutual TLS may be hard for some customers due to their IT infrastructure policies. To enable customers to use their own connectivity solution, Flare now supports **BYOConn** (bring your own connectivity). 

Customer provided connectivity solution must meet the following requirements:

* A client must be able to reach the server. The client and the server do not have to be connected directly. Any number of proxies can be used to provide the communication path between the client and the server.
* The communication path between the client and the server must ensure the confidentiality and integrity of messages.

Since the connectivity solution may expose the server to the internet and allow anyone to connect to the server, it is important to ensure that messages sent to the server are explicitly authenticated.

Flare supports explicit message authentication: messages received by the server must have an auth token, and the token must be validated successfully to prove that it was issued by the server.


Here is how it works:

* The very first thing after the client is started is to login to the server. The server and the client authenticate each other explicitly with the credentials in their startup kits. This step is independent of how the client and the server are connected (mTLS or normal TLS).

* If the client credential is validated successfully, the server issues a token and a signature that binds the client name and token together. The signature is generated with the server's private key to prove that the signature can only be issued by the server.

* When sending a message to the server, the client adds its name, token and the signature as headers to the message.

* When the message is received, the server validates the token and the signature. Messages that are missing these headers or fail to validate will be rejected.


Note that this mechanism is based on the security of the startup kits: login processing and token signature are performed using the PKI credentials in the startup kits of the sites. All sites must protect their startup kits securely, and never share the received token and signature with others.


Message authentication applies to ALL messages that go through the Server, regardless of how the message is initiated (by FL clients, 3rd-party systems using client API, and even by the server itself). 

-------
## Provision Connection Security


The Provision system now allows you to specify connection security explicitly for each site. 


**TLS**

This is normal TLS (i.e. 1-way SSL). While client certificates are not required for establishing the connection, a Root Certificate is required to validate the server. You can provide a custom root cert for validating the server (usually the endpoint that represents the server, and the server is actually behind that endpoint). If you do not provide a custom root certificate, the root cert generated by the Provision tool will be used.



**mTLS**


This is mutual TLS (i.e. 2-way SSL). PKI credentials in the startup kits will be used for client/server connections.


>TLS and mTLS use two different ways for authenticating the peers. In TLS, client authenticates the server, but the server does not authenticate the client. TLS is also called one-way SSL. During the SSL handshake, server has to present evidence signed with its private key; and the client has to be able to validate the evidence with the server's public key (usually contained in the server's certificate). But the client does not need to present anything to the server.

In mTLS, client and server authenticate each other, meaning that they must present evidence signed with their private key and they must be able to validate the evidence with each other's public key.

So both sites using TLS doesn't mean mTLS. In fact, both sites must use the same mode (mTLS or TLS) at the same time.


**Clear**


Messages are not encrypted, and this is usually used when the server is deployed behind a proxy, and the communication between the proxy and the server is in clear text.


### Configuration


To configure connection security, you need to define the connection_security
and the custom_ca_cert property.  You can define them at the project level and participant level. 

Here is a snippet that shows how to use these properties:

```yaml

api_version: 3
name: test25
description: NVIDIA FLARE sample project yaml file
connection_security: tls
custom_ca_cert: /path/of/customRoot.pem


participants:
 # change example.com to the FQDN of the server
 - name: server
   type: server
   org: nvidia
   fed_learn_port: 8002
   admin_port: 8003
   connection_security: clear
 - name: site-1
   type: client
   org: nvidia

```

In this example, the default connection security for the project is “tls” since it’s specified at the project level.  However the server is configured to be “clear” since it is behind a secure proxy. All clients will use “tls” to connect to the proxy. The “custom_ca_cert” is actually the cert for the clients to validate the proxy server.

The custom_ca_cert property is only used for server authentication when making a TLS connection. If not specified, the root CA cert generated by the Provision System will be used.

If connection_security is not specified, the default will be mTLS.

### When to Use Custom CA Certificates

Customers may implement their connectivity using some proxies that will sit between FL Clients and the FL Server. FL Clients only directly connect to the proxy server, which then connects to the FL Server. Typically FL Clients will use one-way SSL to connect to the proxy server. In this case, the custom_ca_cert is the CA cert used by the proxy server.


-----------
## Multi-Address Support of FL Server

Currently, the FL Server uses a single address that all FL clients must use to access it. Some customers find it to be very limiting, and they want to be able to use different addresses for different clients. Another limitation is that the server address must be specified as a domain name - IP addresses are not supported.  Furthermore, the domain name also has to be specified as the “name” of the server in Provision config. Since the name is set to the “common name” of the server’s certificate, it cannot exceed 63 characters. This makes it impossible to use domain names longer than 63 characters.

Flare 2.6 will support multiple addresses for the FL Server. In this case, the FL Server can expose multiple addresses that can be used for FL clients to connect. Depending on the customer’s IT policies, different FL clients may use different addresses. For example, the FL Server may provide two addresses, one accessible from the internet, another accessible only from the internal network.

Server address can be specified as a domain name or an IP address.

Example

```yaml

participants:
 # change example.com to the FQDN of the server
 - name: server
   type: server
   org: nvidia
   fed_learn_port: 8002
   admin_port: 8003
   host_names: [localhost, 127.0.0.1]
   default_host: localhost
   # connection_security: insecure
 - name: red
   type: client
   org: nvidia
   connect_to: 127.0.0.1
 - name: blue
   type: client
   org: nvidia
   connect_to: localhost
 - name: silver
   type: client
   org: nvidia
 - name: admin@nvidia.com
   type: admin
   org: nvidia
   role: project_admin
   connect_to: 127.0.0.1

```

In this example, the FL Server defines additional two host names using the host_names property: localhost (a domain name) and 127.0.0.1 (an IP address). To be backward compatible, the name of the server is treated as the default address if the “default_host” property is not explicitly defined. In this example, the default address is explicitly defined as “localhost”.

Addresses specified with the host_names property are limited to 255 characters.

Three clients are defined here: red, blue, and silver. The “connect_to” property specifies the address to use for the client. Of course, the specified address must be available from the server.

In this example, client “red” will connect to 127.0.0.1; client “blue” will connect to “localhost”; client “silver” will connect to the default address of the server, which is “localhost”.

The admin client will connect to 127.0.0.1.

For this configuration to work, the IT Administrator of the FL Server must ensure that the specified addresses are actually accessible.

## Connection Security and secure_train Flag

Connection Security specifies how the connection will be made (Clear, TLS or mTLS), which is independent of whether training is secure mode or not. If connection security is not explicitly specified for a site, then its connection security will be decided by the “secure_train” flag. If secure_train is True, then use mTLS, otherwise use clear. This is to be backward compatible.   In the past, these two things were treated as one.

The secure_train flag applies to the whole project - you cannot have the case that this flag is True for some sites and False for another.

Connection Security is site specific - different sites can use different settings as long as connections can be made properly. For example, the Server may define connection security to be “clear”, whereas Site-1 may define it to be TLS, and Site-2 may define it to be mTLS. You may wonder how this could ever work. It of course won’t work if the two sites connect to the Server directly. But in the case of BYOConn, you never know how the customer will set up their communication network. For example, if they use proxies between the Server and the sites, the sites won’t directly connect to the Server. The customer will have to set up the connection security properly so that the sites can connect to their proxies properly.

The secure_train flag also triggers privacy protection features (e.g. loading privacy resources from the site’s local folder). This flag is always set to True, except when using the Simulator.


